In [ ]:
#importações
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Carrega dados e metadados(cats_vs_dogs do TensorFlow)
splits = ["train[:80%]", "train[80%:90%]", "train[90%:100%]"] #define divisao dos dados para treinamento, teste e validação
train_ds, val_ds, test_ds = tfds.load("cats_vs_dogs",
                                       split=splits,
                                       as_supervised=True, #imagem, rótulo
                                       with_info=False)

In [ ]:
IMG_SIZE = (160, 160) #definição da dimensao da imagem
BATCH_SIZE = 32 #definição de lotes de imagens

#pré-processamento da imagem
def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

#faz ajustes aleatórios nas imagens pro modelo "aprender" com mais eficiência
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    return image, label

# preparação do conjunto de treino
train_ds = (train_ds.map(preprocess)
                    .map(augment)
                    .shuffle(1000)
                    .batch(BATCH_SIZE)
                    .prefetch(tf.data.AUTOTUNE))

# preparação do conjunto de validação(augment nao foi utilizada augmentação, para focar na avaliação de imagens reais, sem distorções)
val_ds = (val_ds.map(preprocess)
                  .batch(BATCH_SIZE)
                  .prefetch(tf.data.AUTOTUNE))


In [ ]:
#definição do modelo base com MobileNetV2(transfer learning)
base_model = tf.keras.applications.MobileNetV2(input_shape=(*IMG_SIZE, 3), #define o tamanho de entrada da imagem e canais de cor(3 ou RGB)
                                               include_top=False, # remove a parte final do modelo original, que fazia classificação de 1000 classes do ImageNet.
                                               weights="imagenet")

base_model.trainable = False #congela modelo base

#definição do modelo final(sequencial)
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(), #Reduz as saídas do base_model para um vetor pequeno.
    layers.Dense(1, activation="sigmoid") #definição da última camada com apenas 1 neurônio, para fazer a classificação binária
])


In [ ]:
#compilação e treinamento do modelo

model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

history = model.fit(train_ds,
                    epochs=5,
                    validation_data=val_ds)

In [ ]:
#Fine-Tuning

base_model.trainable = True #descongela as últimas camadas
fine_tune_at = 100  #definição do ponto de corte das camadas

#congela apenas as camadas iniciais(até 100)
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

#recompilação
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss="binary_crossentropy",
              metrics=["accuracy"])

#treinamento com Fine Tuning
history_ft = model.fit(train_ds,
                       epochs=10,
                       validation_data=val_ds,
                       initial_epoch=history.epoch[-1]) #garante que O treinamento continue de onde parou

In [ ]:
#Visualização dos resultados

# Junta os históricos anteriores
acc = history.history["accuracy"] + history_ft.history["accuracy"]
val_acc = history.history["val_accuracy"] + history_ft.history["val_accuracy"]

loss = history.history["loss"] + history_ft.history["loss"]
val_loss = history.history["val_loss"] + history_ft.history["val_loss"]

epochs_range = range(len(acc))

# Plotando
plt.figure(figsize=(14, 5))

# Acurácia
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Treino")
plt.plot(epochs_range, val_acc, label="Validação")
plt.axvline(x=history.epoch[-1], color="gray", linestyle="--", label="Início do Fine-Tuning")
plt.title("Acurácia durante o treino")
plt.xlabel("Época")
plt.ylabel("Acurácia")
plt.legend()

# Perda
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Treino")
plt.plot(epochs_range, val_loss, label="Validação")
plt.axvline(x=history.epoch[-1], color="gray", linestyle="--", label="Início do Fine-Tuning")
plt.title("Perda (loss) durante o treino")
plt.xlabel("Época")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Teste final com imagem nova (cachorro)
img = image.load_img("/content/dog.jpg", target_size=IMG_SIZE)
img_array = image.img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Faz a previsão
prediction = model.predict(img_array)

# probabilidade da classe "Dog"
score = prediction[0][0]

# Calcula probabilidades das duas classes
prob_cat = 1 - score
prob_dog = score

# Define a classe final prevista
class_names = ["Cat", "Dog"]
predicted_class = class_names[int(score > 0.5)]

# Exibe resultados
print(f"Predição: {predicted_class}")
print(f"Probabilidade de Cat: {prob_cat:.4f}")
print(f"Probabilidade de Dog: {prob_dog:.4f}")



In [ ]:
# Teste final com imagem nova (gato)
img = image.load_img("/content/cat.jpg", target_size=IMG_SIZE)
img_array = image.img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Faz a previsão
prediction = model.predict(img_array)

# probabilidade da classe "Dog"
score = prediction[0][0]

# Calcula probabilidades das duas classes
prob_cat = 1 - score
prob_dog = score

# Define a classe final prevista
class_names = ["Cat", "Dog"]
predicted_class = class_names[int(score > 0.5)]

# Exibe resultados
print(f"Predição: {predicted_class}")
print(f"Probabilidade de Cat: {prob_cat:.4f}")
print(f"Probabilidade de Dog: {prob_dog:.4f}")